In [1]:
import pandas as pd
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

df = pd.read_csv('Train.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [4]:
df = df[['text']]

In [5]:
def preprocess_text(row):
    # Convert to lowercase
    text = row.lower()

    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words and punctuation
    stop_words = set(stopwords.words('english'))
    preprocessed_text = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    
    return preprocessed_text

In [6]:
df['tokenized_sents'] = df['text'].apply(preprocess_text)

df.head()

,text,tokenized_sents
0,I grew up (b. 1965) watching and loving the Th...,"[grew, b, 1965, watching, loving, thunderbirds..."
1,"When I put this movie in my DVD player, and sa...","[put, movie, dvd, player, sat, coke, chips, ex..."
2,Why do people who do not know what a particula...,"[people, know, particular, time, past, like, f..."
3,Even though I have great interest in Biblical ...,"[even, though, great, interest, biblical, movi..."
4,Im a die hard Dads Army fan and nothing will e...,"[im, die, hard, dads, army, fan, nothing, ever..."


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample input text
input_text = input("Enter your query: ")

# Tokenized sentences from your dataset
tokenized_sentences = df['tokenized_sents'].tolist()

# Convert tokenized sentences into strings
tokenized_sentences = [" ".join(tokens) for tokens in tokenized_sentences]

# Convert input text into a string if it's a list
if isinstance(input_text, list):
    input_text = " ".join(input_text)

# Convert tokenized sentences and input text into numerical vectors using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized_sentences + [input_text])

# Compute cosine similarity between input text vector and each vector in the dataset
cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

# Get indices of most similar items
most_similar_indices = cosine_similarities.argsort()[0][::-1][:5]  # Get top 5 indices

# Create a DataFrame to store top 5 similar products and their cosine similarities
top_similar_df = pd.DataFrame(columns=['Title', 'Cosine Similarity'])

# Fill the DataFrame with data
for idx in most_similar_indices:
    title = df.loc[idx, 'text']
    similarity = cosine_similarities[0][idx]
    top_similar_df.loc[len(top_similar_df)] = [title, similarity]

# Display the DataFrame
print(f"\nTop 5 similar products for {input_text}:")
print(top_similar_df)


Top 5 similar products for terrible:
                                               Title  Cosine Similarity
0  i really wanted this to be good as i am from L...           0.528075
1  This movie had terrible acting, terrible plot,...           0.500774
2  Remember those terrible war movies your grandm...           0.494657
3  The worst movie i've seen in years (and i've s...           0.466910
4  The worst movie I have seen in a while. Yeah i...           0.453234
